<a href="https://colab.research.google.com/github/yhc907/bt/blob/master/Bloomberg_Data_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from xbbg import blp, pipeline

# Reference Data `BDP` and `BDS`

In [ ]:
blp.bdp('AAPL US Equity', flds=['Security_Name', 'Last_Price'])

,security_name,last_price
AAPL US Equity,Apple Inc,425.04


In [ ]:
blp.bdp('6758 JP Equity', flds='Crncy_Adj_Mkt_Cap', Eqy_Fund_Crncy='USD')

,crncy_adj_mkt_cap
6758 JP Equity,"96,223.65"


In [ ]:
holders = blp.bds('AMZN US Equity', flds='All_Holders_Public_Filings', cache=True)
(
    holders
    .loc[:, ~holders.columns.str.contains(
        f'holder_id|portfolio_name|change|number|'
        f'metro|percent_of_portfolio|source'
    )]
    .rename(
        index=lambda tkr: tkr.replace(' Equity', ''),
        columns={
            'holder_name_': 'holder',
            'position_': 'position',
            'filing_date__': 'filing_dt',
            'percent_outstanding': 'pct_out',
            'insider_status_': 'insider',
        }
    )
).head()

,holder,position,filing_dt,insider,pct_out,institution_type_,country
AMZN US,Bezos Jeffrey P,"55,488,770.00",2020-06-30,Y,11.08,Unclassified,n/a
AMZN US,Advisor Group Holdings Inc,"35,429,155.00",2020-03-31,N-P,7.07,Investment Advisor,United States
AMZN US,Vanguard Group Inc/The,"32,975,915.00",2020-03-31,N-P,6.58,Investment Advisor,United States
AMZN US,BlackRock Inc,"27,023,092.00",2020-03-31,N-P,5.40,Investment Advisor,United States
AMZN US,Scott MacKenzie,"19,786,240.00",2019-08-01,N-P,3.95,Unclassified,n/a


In [ ]:
blp.dividend('SPY US Equity', start_date='2019')

,dec_date,ex_date,rec_date,pay_date,dvd_amt,dvd_freq,dvd_type
SPY US Equity,2020-06-18,2020-06-19,2020-06-22,2020-07-31,1.37,Quarter,Income
SPY US Equity,2020-03-19,2020-03-20,2020-03-23,2020-04-30,1.41,Quarter,Income
SPY US Equity,2019-12-19,2019-12-20,2019-12-23,2020-01-31,1.57,Quarter,Income
SPY US Equity,2019-09-19,2019-09-20,2019-09-23,2019-10-31,1.38,Quarter,Income
SPY US Equity,2019-06-20,2019-06-21,2019-06-24,2019-07-31,1.43,Quarter,Income
SPY US Equity,2019-03-14,2019-03-15,2019-03-18,2019-04-30,1.23,Quarter,Income


In [ ]:
blp.earning('FB US Equity', Eqy_Fund_Year=2018, Number_Of_Periods=2)

,segment_name,level,fy2019,fy2018,fy2019_pct,fy2018_pct
FB US Equity,US & Canada,1,"32,206.00","25,727.00",45.55,46.07
FB US Equity,Europe,1,"16,826.00","13,631.00",23.80,24.41
FB US Equity,Asia-Pacific,1,"15,406.00","11,733.00",21.79,21.01
FB US Equity,Rest of World,1,"6,259.00","4,747.00",8.85,8.50


# Historical Data

Historical data with Excel compatible overrides

In [ ]:
blp.bdh(
    tickers='SHCOMP Index', flds=['high', 'low', 'last_price'],
    start_date='2019-11', end_date='2020', Per='W', Fill='P', Days='A',
)

SHCOMP Index                    
                   high      low last_price
2019-11-01     2,980.13 2,917.15   2,958.20
2019-11-08     3,008.31 2,962.84   2,964.18
2019-11-15     2,949.96 2,891.20   2,891.34
2019-11-22     2,933.99 2,873.99   2,885.29
2019-11-29     2,915.04 2,858.58   2,871.98
2019-12-06     2,912.01 2,857.32   2,912.01
2019-12-13     2,969.98 2,902.79   2,967.68
2019-12-20     3,039.38 2,958.71   3,004.94
2019-12-27     3,036.11 2,960.43   3,005.03

Dividend / split adjustments

In [ ]:
pd.concat([
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=True, CshAdjAbnormal=True, CapChg=True
    ).rename(columns={'Px_Last': 'Px_Adj'}),
    blp.bdh(
        'AAPL US Equity', 'Px_Last', '20140605', '20140610',
        CshAdjNormal=False, CshAdjAbnormal=False, CapChg=False
    ).rename(columns={'Px_Last': 'Px_Raw'}),
], axis=1)

AAPL US Equity       
                   Px_Adj Px_Raw
2014-06-05          83.76 647.35
2014-06-06          83.53 645.57
2014-06-09          84.86  93.70
2014-06-10          85.36  94.25

# Intraday Bars

In [ ]:
cur_dt = pd.Timestamp('today', tz='America/New_York')
recent = pd.bdate_range(end='today', periods=2, tz='America/New_York')
pre_dt = max(filter(lambda dd: dd < cur_dt, recent))
pre_dt.date()

datetime.date(2020, 7, 31)

In [ ]:
blp.bdib('QQQ US Equity', dt=pre_dt, session='day').tail()

2020-08-02 19:58:52,488:xbbg.io.storage.save_intraday:INFO:saving data to C:/Data/Bloomberg/Equity/QQQ US Equity/TRADE/2020-07-31.parq ...


QQQ US Equity                                                      
                                   open   high    low  close   volume num_trds          value
2020-07-31 15:55:00-04:00        265.11 265.25 265.06 265.25   141216      641  37,446,796.00
2020-07-31 15:56:00-04:00        265.24 265.30 265.20 265.27   270704     1124  71,808,048.00
2020-07-31 15:57:00-04:00        265.27 265.37 265.20 265.36   216632      901  57,474,260.00
2020-07-31 15:58:00-04:00        265.36 265.39 265.33 265.34   257720     1314  68,386,880.00
2020-07-31 15:59:00-04:00        265.36 265.88 265.36 265.79  1351384     3826 359,012,960.00

In [ ]:
blp.bdib('388 HK Equity', dt=pre_dt, session='am_open_7')

2020-08-02 19:58:56,071:xbbg.io.storage.save_intraday:INFO:saving data to C:/Data/Bloomberg/Equity/388 HK Equity/TRADE/2020-07-31.parq ...


388 HK Equity                                                   
                                   open   high    low  close volume num_trds         value
2020-07-31 09:30:00+08:00        366.60 367.80 366.00 367.20  87800      279 32,196,540.00
2020-07-31 09:31:00+08:00        367.80 369.60 367.60 369.40  78900      255 29,081,540.00
2020-07-31 09:32:00+08:00        369.40 369.80 368.80 369.20  46300      139 17,100,180.00
2020-07-31 09:33:00+08:00        369.60 370.00 369.20 369.60  31500      108 11,645,980.00
2020-07-31 09:34:00+08:00        370.00 370.00 369.20 369.60  26100       88  9,645,980.00
2020-07-31 09:35:00+08:00        369.60 370.40 369.60 370.00  98900      130 36,589,600.00
2020-07-31 09:36:00+08:00        370.20 370.20 369.80 370.00  64400      114 23,831,420.00
2020-07-31 09:37:00+08:00        370.20 370.60 370.00 370.60  32400       84 11,996,560.00

# Intraday Tick Data

In [ ]:
blp.bdtick('QQQ US Equity', dt=pre_dt).tail(10)

QQQ US Equity                        
                                    typ  value volume cond exch
2020-07-31 20:00:00-04:00         TRADE 265.84      0   NC    P
2020-07-31 20:00:00-04:00         TRADE 265.78      0   OC    N
2020-07-31 20:00:00-04:00         TRADE 264.76      0   OC    M
2020-07-31 20:00:00-04:00         TRADE 265.84      0   OC    K
2020-07-31 20:00:00-04:00         TRADE 265.82      0   OC    J
2020-07-31 20:00:00-04:00         TRADE 265.67      0   OC    C
2020-07-31 20:00:00-04:00         TRADE 265.85      0   OC    B
2020-07-31 20:00:00-04:00         TRADE 265.60      0   OC    A
2020-07-31 20:00:00-04:00         TRADE 265.76      0   OC    D
2020-07-31 20:00:00-04:00         TRADE 265.79      0   CC    Q

# Equity Screen `BEQS`

In [ ]:
blp.beqs('Core Capital Ratios', typ='GLOBAL').head().iloc[:, :6]

,ticker,short_name,market_cap,tier_1_cap_rt_lf,tot_rsk_bsd_cap_rt_lf,moody's_issuer_rtg
8331 JP,8331 JP,CHIBA BANK LTD,"3,713,918,720.00",11.51,12.12,A1
8355 JP,8355 JP,SHIZUOKA BANK,"3,846,072,064.00",15.59,15.59,A1
ABN NA,ABN NA,ABN AMRO BANK-CV,"7,785,078,272.00",19.10,25.20,A1
ACA FP,ACA FP,CREDIT AGRICOLE,"27,639,259,136.00",12.90,16.70,Aa3
BAC US,BAC US,BANK OF AMERICA,"215,562,747,904.00",13.20,15.80,A2


# Subscription

`blp.live` will yield market data as dict.
If the data is required to pass as json, use `json=True` in `kwargs`

In [ ]:
for snap in blp.live(['ESA Index', 'NQA Index'], max_cnt=10):
    print(snap)

{'TICKER': 'ESA Index', 'MKTDATA_EVENT_TYPE': 'SUMMARY', 'MKTDATA_EVENT_SUBTYPE': 'INITPAINT', 'BID': 3268.25, 'ASK': 3269.5, 'BEST_BID': 3268.25, 'BEST_ASK': 3269.5, 'IND_BID_FLAG': False, 'IND_ASK_FLAG': False, 'ASK_SIZE': 2, 'BID_SIZE': 6, 'SIMP_LAST_PX_ALL_SESS_DIR_RT': 1, 'ID_BB_SEC_NUM_SRC': 2078868620302, 'LAST_PRICE': 3263.5, 'SIZE_LAST_TRADE': 1, 'LAST_TRADE': 3269.5, 'VOLUME': 2225251, 'HIGH': 3273.75, 'LOW': 3212.5, 'OPEN': 3269.25, 'BID_YIELD': 3425.5, 'ASK_YIELD': 3101.5, 'RT_OPEN_INTEREST': 2613918, 'RT_EXCH_MARKET_STATUS': 'ACTV', 'MIN_LIMIT': 3100.0, 'DOWN_LIMIT': 3100.0, 'MAX_LIMIT': 3427.0, 'UP_LIMIT': 3427.0, 'RT_TRADING_PERIOD': 'CLOS', 'PREV_CLOSE_VALUE_REALTIME': 3263.5, 'TRADING_DT_REALTIME': datetime.date(2020, 8, 3), 'RT_OPEN_INT_DT': datetime.date(2020, 7, 30), 'PREV_TRADING_DT_REALTIME': datetime.date(2020, 7, 31), 'PRICE_CHANGE_1Y_NET_RT': 271.75, 'PRICE_CHANGE_1Y_PCT_RT': 9.079999923706055, 'CLOSING_PRICE_1Y_AGO_RT': 2991.75, 'PX_SETTLE_ACTUAL_RT': 3263.5, 

# Pipelines

In [ ]:
cur_dt = pd.Timestamp('today', tz='America/New_York')
recent = pd.bdate_range(end='today', periods=2, tz='America/New_York')
pre_dt = max(filter(lambda dd: dd < cur_dt, recent))

fx = blp.bdib('JPY Curncy', dt=pre_dt)
jp = pd.concat([
    blp.bdib(ticker, dt=pre_dt, session='day')
    for ticker in ['7974 JP Equity', '9984 JP Equity']
], axis=1)
jp.tail()

2020-08-02 20:00:09,932:xbbg.io.storage.save_intraday:INFO:saving data to C:/Data/Bloomberg/Curncy/JPY Curncy/TRADE/2020-07-31.parq ...
2020-08-02 20:00:10,468:xbbg.io.storage.save_intraday:INFO:saving data to C:/Data/Bloomberg/Equity/7974 JP Equity/TRADE/2020-07-31.parq ...
2020-08-02 20:00:11,403:xbbg.io.storage.save_intraday:INFO:saving data to C:/Data/Bloomberg/Equity/9984 JP Equity/TRADE/2020-07-31.parq ...


7974 JP Equity                                         ... 9984 JP Equity                   \
                                    open      high       low     close   volume  ...            low    close  volume   
2020-07-31 14:54:00+09:00      46,460.00 46,460.00 46,440.00 46,460.00   800.00  ...       6,608.00 6,613.00   66200   
2020-07-31 14:55:00+09:00      46,450.00 46,460.00 46,420.00 46,430.00 7,500.00  ...       6,603.00 6,604.00   98300   
2020-07-31 14:56:00+09:00      46,420.00 46,460.00 46,420.00 46,430.00 4,500.00  ...       6,600.00 6,604.00   73000   
2020-07-31 14:57:00+09:00      46,440.00 46,460.00 46,440.00 46,450.00 2,800.00  ...       6,600.00 6,604.00   89400   
2020-07-31 14:58:00+09:00      46,440.00 46,480.00 46,440.00 46,480.00 8,000.00  ...       6,601.00 6,608.00  110300   

                                                   
                          num_trds          value  
2020-07-31 14:54:00+09:00      124 437,683,904.00  
2020-07-31 14:55:00+09:00      188 649,790,784.00  
2020-07-31 14:56:00+09:00      170 482,047,392.00  
2020-07-31 14:57:00+09:00      158 590,299,968.00  
2020-07-31 14:58:00+09:00      181 728,525,184.00  

[5 rows x 14 columns]

Get `close` prices and convert to USD

In [ ]:
prices = (
    jp
    .pipe(pipeline.get_series, col='close')
    .pipe(pipeline.apply_fx, fx=fx)
    .tz_convert('Asia/Tokyo')
)
prices.tail()

,7974 JP Equity,9984 JP Equity
2020-07-31 14:54:00+09:00,445.23,63.37
2020-07-31 14:55:00+09:00,444.90,63.28
2020-07-31 14:56:00+09:00,444.90,63.28
2020-07-31 14:57:00+09:00,445.14,63.29
2020-07-31 14:58:00+09:00,445.38,63.32


## Customized Pipelines

VWAP for intraday bar data

In [ ]:
def vwap(data: pd.DataFrame, fx=None, name=None) -> pd.Series:
    return pd.Series({
        ticker: (
            data[ticker][['close', 'volume']].prod(axis=1).sum()
            if fx is None else (
                data[ticker].close
                .pipe(pipeline.apply_fx, fx)
                .close
                .mul(data[ticker].volume)
                .sum()
            )
        ) / data[ticker].volume.sum()
        for ticker in data.columns.get_level_values(0).unique()
    }, name=name)

VWAP in local currency

In [ ]:
jp.pipe(vwap, name=jp.index[-1].date())

7974 JP Equity   46,700.20
9984 JP Equity    6,785.66
Name: 2020-07-31, dtype: float64

VWAP in USD

In [ ]:
jp.pipe(vwap, fx=fx, name=jp.index[-1].date())

7974 JP Equity   447.38
9984 JP Equity    64.99
Name: 2020-07-31, dtype: float64

Total traded volume as of time in day for past few days

In [ ]:
jp_hist = pd.concat([
    pd.concat([
        blp.bdib(ticker, dt=dt, session='day')
        for ticker in ['7974 JP Equity', '9984 JP Equity']
    ], axis=1)
    for dt in pd.bdate_range(end='today', periods=10)[:-1]
], sort=False)
unique(jp_hist.index.date)

array([datetime.date(2020, 7, 20), datetime.date(2020, 7, 21),
       datetime.date(2020, 7, 22), datetime.date(2020, 7, 27),
       datetime.date(2020, 7, 28), datetime.date(2020, 7, 29),
       datetime.date(2020, 7, 30)], dtype=object)

In [ ]:
def drop_zeros(data: pd.DataFrame) -> pd.DataFrame:
    return (
        data
        .replace(0, np.nan)
        .dropna(how='all')
        .replace(np.nan, 0)
    )

In [ ]:
def traded_volume(data: pd.DataFrame, asof: str) -> pd.DataFrame:
    return (
        data
        .pipe(pipeline.get_series, col='volume')
        .between_time('0:00', asof)
        .resample('B')
        .sum()
        .pipe(drop_zeros)
    )

In [ ]:
jp_hist.pipe(traded_volume, asof='10:00')

,7974 JP Equity,9984 JP Equity
2020-07-20 00:00:00+09:00,"289,600.00","5,112,200.00"
2020-07-21 00:00:00+09:00,"668,900.00","8,102,200.00"
2020-07-22 00:00:00+09:00,"229,400.00","7,589,500.00"
2020-07-27 00:00:00+09:00,"250,500.00","7,051,600.00"
2020-07-28 00:00:00+09:00,"211,600.00","6,751,300.00"
2020-07-29 00:00:00+09:00,"165,900.00","6,574,800.00"
2020-07-30 00:00:00+09:00,"166,400.00","7,945,300.00"


In [ ]:
jp_hist.pipe(traded_volume, asof='11:00')

,7974 JP Equity,9984 JP Equity
2020-07-20 00:00:00+09:00,"421,500.00","13,141,100.00"
2020-07-21 00:00:00+09:00,"788,700.00","13,217,700.00"
2020-07-22 00:00:00+09:00,"355,600.00","10,967,200.00"
2020-07-27 00:00:00+09:00,"349,600.00","10,750,700.00"
2020-07-28 00:00:00+09:00,"280,900.00","9,959,300.00"
2020-07-29 00:00:00+09:00,"236,800.00","9,270,700.00"
2020-07-30 00:00:00+09:00,"230,500.00","11,454,800.00"
